# Subpace Overlap

---

The subspace overlap is but one of the measures used to compare the subspaces determined by both the Elastic Network Model (ENM) and Principal Component Analysis (PCA). PCA is a powerful tool to quantify the extent to which two protein simulations or fragements thereof (__A__ and __B__) explore the same conformational space. To do so, a common approach is to select a subset of eigenvectors for each ensemble, e.g., $\pmb{v}_{1}^{A},\dots,\pmb{v}_{n}^{A}$ and $\pmb{v}_{1}^{B},\dots,\pmb{v}_{n}^{B}$. Thus, we can use the following expression:

\begin{align}
\Psi_{A:B} = \frac{1}{n} \sum_{i=1}^{n} \sum_{j=1}^{n} (\pmb{v}_{i}^{A} \pmb{\cdot} \, \pmb{v}_{j}^{B})^{2}
\end{align}

The _subspace overlap_ $\Psi$ ranges from $0$, when the eigenvector subsets are completely dissimilar, to $1$ (or $100\%)$ when they are identical. The number of eigenvectors used, $n$, is typically chosen so as to represent a significant proportion of the fluctuations in the simulation.

Knowing this, let us calculate the subspace overlap between two trajectories.

---

In [8]:
# Author: Jan A. Siess
# Date: 06 September 2019
#
# Purpose: To calculate the subspace overlap between two trajectories in order
# to determine the degree similarity between two proteins spanning a similar
# conformational space.

from prody import *
from pylab import *
import numpy as np


### For Trajectory A

First, we must conduct the ANM. Toward the end, we will receive a set of eigenvectors with which we will be able to use.

---

In [9]:
# Python Version: Intended for Python versions 3.[*]+
protA = 'snn_SC.nowa.avg'

#if in regular terminal working with a regular [*].py file:
#prot = sys.argv[1]

# Parse structure of PDB by passing an identifier. If it is not located in the current working --
# directory, it will be downloaded.
avgA = parsePDB('snn_SC.nowa.avg.pdb')

#Extracting only C$\alpha$ atoms.
calphas_protA = avgA.select('protein and name CA')

#Get the actual number of alpha carbon atoms into a variable
num_CA_protA = calphas_protA.numAtoms('protein')

print(f'The number of Carbon Atoms is: {num_CA_protA}')

@> 739 atoms and 1 coordinate set(s) were parsed in 0.01s.
The number of Carbon Atoms is: 47


In [10]:
#Building the Hessian

#Instantiate an ANM instance:
anm = ANM('avg ANM analysis')

#Build the Hessian matrix by passing the selected @>568 atoms to ANM.buildHessian() method:
anm.buildHessian(calphas_protA, cutoff = 13.5, gamma = 1.0)

#Get a copy of the Hessian matrix using ANM.getHessian() method [further, we limit full output to only round up to 3 places]:
structHessian = anm.getHessian().round(3)

#Just out of curiousity to determine the number of element in the array:
SquareMatrix = np.prod(structHessian.shape)
print(f'The number of array elements in the Hessian matrix amounts to: {SquareMatrix}')

#Setting the force constant K to 1.0 kcal/Å^{2} and the cutt-off distance to 13.5 Å 
#This was determined from previously derived general ranges for proteins.
CO = 13.5
K = 1*10**0

#Save the array to a text file 
np.savetxt(f'{protA}_Hessian_gamma_{K}_cutoff_{CO}.dat', structHessian)

@> Hessian was built in 0.02s.
The number of array elements in the Hessian matrix amounts to: 19881


In [11]:
#Obtaining the normal modes data 

#The normal mode of an oscillating system is a pattern of motion in which all parts of the system move sinusoidally with the same frequency and with a fixed phase relation.

#n_modes -- number of non-zero eigenvalues/vectors to calculate
#zeros -- if True, modes with zero eigencalues will be kept
#turbo -- Use a memory intensive, but faster way to calculate modes

#determine first the number of n_modes
num_modes_protA = num_CA_protA*3 

#Go ahead with the main calculation...

#Calculating Normal Modes, eigenvalues/-vectors, and the covariance matrix:
anm.calcModes(n_modes=num_modes_protA, zeros = False, turbo = True)
eigVals_protA = anm.getEigvals()
eigVects_protA = anm.getEigvecs()
egCov_protA = anm.getCovariance()

#Save the eigenvalue/-vectors into a txt file:
np.savetxt(f'{protA}_egvals2_gamma_{K}_cutoff_{CO}.dat', eigVals_protA)
np.savetxt(f'{protA}_egvects2_gamma_{K}_cutoff_{CO}.dat', eigVects_protA)
np.savetxt(f'{protA}_egCov2_gamma_{K}_cutoff_{CO}.dat', egCov_protA)

@> 135 modes were calculated in 0.02s.


### For Trajectory B

Again, we must conduct the ANM. Toward the end, we will receive a set of eigenvectors with which we will be able to use. This time, however, we will be executing this all for the second trajectory with which we would like to compare the first to. 

---

In [12]:
# Python Version: Intended for Python versions 3.[*]+
protB = 'sn_homodimer.nowa.avg'

#if in regular terminal working with a regular [*].py file:
#prot = sys.argv[1]

# Parse structure of PDB by passing an identifier. If it is not located in the current working --
# directory, it will be downloaded.
avgB = parsePDB('sn_homodimer.nowa.avg.pdb')

#Extracting only C$\alpha$ atoms.
calphas_protB = avgB.select('protein and name CA')

#Get the actual number of alpha carbon atoms into a variable
num_CA_protB = calphas_protB.numAtoms('protein')

print(f'The number of Carbon Atoms is: {num_CA_protB}')

@> 698 atoms and 1 coordinate set(s) were parsed in 0.01s.
The number of Carbon Atoms is: 44


In [14]:
#Building the Hessian

#Instantiate an ANM instance:
anm = ANM('avg ANM analysis')

#Build the Hessian matrix by passing the selected @>568 atoms to ANM.buildHessian() method:
anm.buildHessian(calphas_protB, cutoff = 13.5, gamma = 1.0)

#Get a copy of the Hessian matrix using ANM.getHessian() method [further, we limit full output to only round up to 3 places]:
structHessian = anm.getHessian().round(3)

#Just out of curiousity to determine the number of element in the array:
SquareMatrix = np.prod(structHessian.shape)
print(f'The number of array elements in the Hessian matrix amounts to: {SquareMatrix}')

#Setting the force constant K to 1.0 kcal/Å^{2} and the cutt-off distance to 13.5 Å 
#This was determined from previously derived general ranges for proteins.
CO = 13.5
K = 1*10**0

#Save the array to a text file 
np.savetxt(f'{protB}_Hessian_gamma_{K}_cutoff_{CO}.dat', structHessian)

@> Hessian was built in 0.02s.
The number of array elements in the Hessian matrix amounts to: 17424


In [15]:
#Obtaining the normal modes data 

#The normal mode of an oscillating system is a pattern of motion in which all parts of the system move sinusoidally with the same frequency and with a fixed phase relation.

#n_modes -- number of non-zero eigenvalues/vectors to calculate
#zeros -- if True, modes with zero eigencalues will be kept
#turbo -- Use a memory intensive, but faster way to calculate modes

#determine first the number of n_modes
num_modes_protB = num_CA_protB*3 

#Go ahead with the main calculation...

#Calculating Normal Modes, eigenvalues/-vectors, and the covariance matrix:
anm.calcModes(n_modes=num_modes_protB, zeros = False, turbo = True)
eigVals_protB = anm.getEigvals()
eigVects_protB = anm.getEigvecs()
egCov_protB = anm.getCovariance()

#Save the eigenvalue/-vectors into a txt file:
np.savetxt(f'{protB}_egvals2_gamma_{K}_cutoff_{CO}.dat', eigVals_protB)
np.savetxt(f'{protB}_egvects2_gamma_{K}_cutoff_{CO}.dat', eigVects_protB)
np.savetxt(f'{protB}_egCov2_gamma_{K}_cutoff_{CO}.dat', egCov_protB)

@> 126 modes were calculated in 0.01s.


## Subspace Overlap Calculation

Using the eigenvectors calculated for each previous trajectory, we can now go ahead and take the dot product between the two, square that, and then take the sum to achieve the subspace overlap. Let us start by taking the first 10 eigenvectors.

In [16]:
print(len(eigVects_protA))
print(len(eigVects_protB))

141
132


In [17]:
#b = eigVects_protB.T
#x = np.dot(eigVects_protA,b)
#print(x)

result = 0

for i in range(0,10):
    dot_prod = np.dot(eigVects_protA[i],eigVects_protB[i])
    squared_result = (dot_prod)**2
    result = result + squared_result
print((result/10)*100) 


ValueError: shapes (135,) and (126,) not aligned: 135 (dim 0) != 126 (dim 0)

# Results 

The subspace overlap between... 

The pig and shrimp is: 0.0958283075352032

The pig and drosophila is: 0.7104850377029857

The shrimp and drosophila is: 0.07553883892467632

In [ ]:
anm1 = calcANM('acc_SC.run1.nowa.pdb', selstr="calpha", cutoff=13.5, gamma=1.0,n_modes=20)
anm2 = calcANM('acc_SC.run2.nowa.pdb', selstr="calpha", cutoff=13.5, gamma=1.0,n_modes=20)
calcSubspaceOverlap(anm1[0],anm2[0])